# Day 3 - Binary Diagnostic

https://adventofcode.com/2021/day/3

In [14]:
from pathlib import Path

INPUTS = Path("input.txt").read_text().strip().split("\n")


## Part 1

Determining the most common bit in each position of all numbers combined essentially means obtaining counts of those bits across all numbers, but I'd like to be a bit clever this time around.

Suppose I transposed all numbers, then sorted the individually transposed columns. After re-transposing, the gamma rate (most common bit) would be equal to whatever number appears halfway in the sequence of inputs, since the "most common" when sorted this way would have to cross that threshold.

In [15]:
from typing import Sequence


def transpose_matrix(matrix: list[Sequence]) -> list[list]:
    """Transposes a matrix, so rows become columns and vice versa."""
    return list(map(list, zip(*matrix)))


transposed = transpose_matrix(INPUTS)
sorted_transposed = [sorted(x) for x in transposed]
resorted = ["".join(x) for x in transpose_matrix(sorted_transposed)]

# finding the middle sequence in the sorted set should produce our gamma rate
gamma_binary = resorted[len(resorted) // 2]
print(f"{gamma_binary=}")

# our epsilon rate is the inverted sequence of the gamma rate (swap 0 for 1 and vice versa)
epsilon_binary = "".join(["1" if x == "0" else "0" for x in gamma_binary])
print(f"{epsilon_binary=}")

# convert to integer numbers:
gamma_rate = int(gamma_binary, base=2)
epsilon_rate = int(epsilon_binary, base=2)
print(f"{gamma_rate=}")
print(f"{epsilon_rate=}")

# Finally, multiply the two together to get power consumption
power_consumption = gamma_rate * epsilon_rate
print(f"{power_consumption=}")


gamma_binary='110111000111'
epsilon_binary='001000111000'
gamma_rate=3527
epsilon_rate=568
power_consumption=2003336


Turns out, that transposing method worked perfectly. 👍🏻

## Part 2

This one makes things slightly trickier. It's a very similar methodology as the original, but we have to continually pare down our list of inputs until only one remains, so we can't do the transpose trick again.

This time what I'll do is separate inputs into buckets based on the bit being checked. Whichever bucket fits the criteria goes on to a new round of sorting until a bucket is checked with only a single string.

In [23]:
def reduce_by_bit_commonality(
    bucket: list[str],
    most_common: bool = True,
    pos: int = 0,
) -> str:
    if not bucket:
        raise ValueError("Can't find anything in an empty bucket")
    if len(bucket) == 1:
        # Final answer!
        return bucket[0]

    bucket_ones = []
    bucket_zeroes = []
    for num in bucket:
        # Filter values from the bucket into the ones or zeroes
        # based on the content of the selected bit
        match num[pos]:
            case '1':
                bucket_ones.append(num)
            case '0':
                bucket_zeroes.append(num)
    if most_common:
        # Prefer the 1s if lengths are equal; otherwise select largest bucket
        new_bucket = bucket_ones if len(bucket_ones) >= len(bucket_zeroes) else bucket_zeroes
    else:
        # Prefer the 0s if lengths are equal; otherwise select smallest bucket
        new_bucket = bucket_zeroes if len(bucket_zeroes) <= len(bucket_ones) else bucket_ones
    
    # recurse with the selected bucket in the next bit position (pos + 1)
    return reduce_by_bit_commonality(new_bucket, most_common=most_common, pos=pos+1)

oxy_binary = reduce_by_bit_commonality(INPUTS, most_common=True)
co2_binary = reduce_by_bit_commonality(INPUTS, most_common=False)
print(f"{oxy_binary=}")
print(f"{co2_binary=}")

# convert to integers
oxy_rate = int(oxy_binary, base=2)
co2_rate = int(co2_binary, base=2)
print(f"{oxy_rate=}")
print(f"{co2_rate=}")

# and get our final life_support answer by multiplying
life_support = oxy_rate * co2_rate
print(f"{life_support=}")


oxy_binary='100111110011'
co2_binary='001011100001'
oxy_rate=2547
co2_rate=737
life_support=1877139
